In [7]:
import tensorflow as tf
import autokeras as ak

## Load MNIST dataset

In [8]:
from tensorflow.keras.datasets import cifar10, fashion_mnist, mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


## ResNetBlock

In [9]:
import timeit

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ResNetBlock()(output_node) # version="v2", pretrained=False
output_node = ak.ClassificationHead()(output_node)

resnet_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
# Use the first 100 training samples for 1 epoch as a quick demo.
# You may run with the full dataset with 10 epochs, but expect a longer training time.
resnet_auto_model.fit(x_train[:100], y_train[:100], epochs=1, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

Trial 3 Complete [00h 01m 30s]
val_loss: 2.6961822509765625

Best val_loss So Far: 4.492432594299316
Total elapsed time: 00h 04m 26s
INFO:tensorflow:Oracle triggered exit
2/2 [==============================] - 74s 24s/step - loss: 34.8798 - accuracy: 0.1400
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Total time: 394.28 seconds.


/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


### Get the summarized results during the tuning process

In [10]:
resnet_auto_model.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
res_net_block_2/pretrained: False
res_net_block_2/version: resnet50
res_net_block_2/imagenet_size: True
classification_head_2/spatial_reduction_4/reduction_type: flatten
classification_head_2/dropout: 0
optimizer: adam
learning_rate: 0.001
classification_head_2/spatial_reduction_5/reduction_type: flatten
classification_head_2/spatial_reduction_8/reduction_type: flatten
Score: 4.492432594299316
Trial summary
Hyperparameters:
res_net_block_2/pretrained: False
res_net_block_2/version: resnet50
res_net_block_2/imagenet_size: True
classification_head_2/spatial_reduction_4/reduction_type: flatten
classification_head_2/dropout: 0
optimizer: adam
learning_rate: 0.1
classification_head_2/spatial_reduction_5/reduction_type: flatten
classification_head_2/spatial_reduction_6/reduction_type: flatten
Score: nan
Trial summary
Hyperparameters:
res_net_block_2/pretra

### Display best model

In [11]:
best_resnet_model = resnet_auto_model.export_model()
best_resnet_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 28, 28)]     0           []                               
                                                                                                  
 cast_to_float32_7 (CastToFloat  (None, 28, 28)      0           ['input_15[0][0]']               
 32)                                                                                              
                                                                                                  
 expand_last_dim_7 (ExpandLastD  (None, 28, 28, 1)   0           ['cast_to_float32_7[0][0]']      
 im)                                                                                              
                                                                                            

### Evaluate the best resnet model on the test data.

In [ ]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



## XceptionBlock

In [ ]:
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.XceptionBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

xception_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
# Use the first 100 training samples for 1 epoch as a quick demo.
# You may run with the full dataset with 10 epochs, but expect a longer training time.
xception_auto_model.fit(x_train[:100], y_train[:100], epochs=1, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

### Display the best xception model

In [ ]:
best_xception_model = xception_auto_model.export_model()
tf.keras.utils.plot_model(best_xception_model, show_shapes=True, expand_nested=True) # rankdir='LR'

In [ ]:
best_xception_model.summary()

### Evaluate the best xception model on the test data.

In [ ]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



## HyperBlock for image classification (ImageBlock)

In [ ]:
import timeit
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.ImageBlock(
                # Normalize the dataset.
                normalize=True,
                # Do not do data augmentation.
                augment=False)(input_node)
output_node = ak.ClassificationHead(dropout=0.0)(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
# Use the first 100 training samples for 1 epoch as a quick demo.
# You may run with the full dataset with 10 epochs, but expect a longer training time.
auto_model.fit(x_train[:100], y_train[:100], epochs=1, batch_size=32) # , batch_size=64
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

In [ ]:
auto_model.tuner.results_summary(num_trials=1)

In [ ]:
best_model = auto_model.export_model()
best_model.summary()

In [ ]:

test_loss, test_acc = auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))

